In [1]:
import matplotlib.pyplot as plt
import numpy as np

import keras

In [6]:
!wget "https://www.dropbox.com/sh/s9r1av3m4eatd3y/AAA8zYti5b5tnyKfcah2Reaja"

--2020-10-17 14:15:32--  https://www.dropbox.com/sh/s9r1av3m4eatd3y/AAA8zYti5b5tnyKfcah2Reaja
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/s9r1av3m4eatd3y/AAA8zYti5b5tnyKfcah2Reaja [following]
--2020-10-17 14:15:32--  https://www.dropbox.com/sh/raw/s9r1av3m4eatd3y/AAA8zYti5b5tnyKfcah2Reaja
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2c1cf29d5b4209535ac0424146.dl.dropboxusercontent.com/zip_download_get/Ak2e2C7A_dbP9NYEww-TntwqWOLSkbBDTyF3bsQn7Dl6SNHbAuFsrMAKoEtrJ1EFzjNDtsn1MTE1xUx2cUvNqZe2ukne4EZvpCs6k1d70829sA [following]
--2020-10-17 14:15:33--  https://uc2c1cf29d5b4209535ac0424146.dl.dropboxusercontent.com/zip_download_get/Ak2e2C7A_dbP9NYEww-TntwqWOLSkbBDTyF3bsQn7Dl6SNHbAuFsrMAKoEtrJ1EFzjNDtsn1MTE1xUx2cUvNqZe2u

In [7]:
!unzip "AAA8zYti5b5tnyKfcah2Reaja" -d dataset/

Archive:  AAA8zYti5b5tnyKfcah2Reaja
mapname:  conversion of  failed
   creating: dataset/Test/
   creating: dataset/Train/
   creating: dataset/Test/Pikachu/
   creating: dataset/Train/Pikachu/
   creating: dataset/Test/Bulbasaur/
   creating: dataset/Test/Charmander/
   creating: dataset/Train/Bulbasaur/
   creating: dataset/Train/Charmander/
 extracting: dataset/Train/Pikachu/348.jpg  
 extracting: dataset/Train/Pikachu/319.jpg  
 extracting: dataset/Test/Pikachu/2032.jpg  
 extracting: dataset/Test/Pikachu/2011.jpg  
 extracting: dataset/Train/Pikachu/378.jpg  
 extracting: dataset/Train/Pikachu/344.jpg  
 extracting: dataset/Train/Pikachu/379.jpg  
 extracting: dataset/Train/Pikachu/363.jpg  
 extracting: dataset/Train/Pikachu/385.jpg  
 extracting: dataset/Train/Pikachu/302.jpg  
 extracting: dataset/Test/Pikachu/2000.jpg  
 extracting: dataset/Train/Pikachu/313.jpg  
 extracting: dataset/Train/Pikachu/309.jpg  
 extracting: dataset/Train/Pikachu/342.jpg  
 extracting: dataset/Tra

## Image Augmentation/ Data Generator/ Data Loader

In [8]:
from keras.preprocessing import image

In [9]:
train_datagen = image.ImageDataGenerator(rotation_range=20, 
                         width_shift_range=0.2, 
                         height_shift_range=0.2, 
                         shear_range =0.2, 
                         zoom_range=0.3, 
                         horizontal_flip = True,
                         rescale = 1/255.
                         )

In [10]:
val_datagen = image.ImageDataGenerator(rescale = 1/255.)

In [11]:
train_generator = train_datagen.flow_from_directory("dataset/Train", target_size=(100,100))

Found 304 images belonging to 3 classes.


In [12]:
train_generator.class_indices

{'Bulbasaur': 0, 'Charmander': 1, 'Pikachu': 2}